<h1 style="background-color:yellow;font-family:newtimeroman;font-size:550%;text-align:center;border-radius: 15px 50px;padding: 5px">Vehicle Identification</h1>


<center><img src="https://i.ytimg.com/vi/uD8kWFaL9_g/maxresdefault.jpg",height='500',width='600'></center>


<h1 style="background-color:green;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">OverView </h1>
<a id=0></a>

In this dataset our main objective is to predict if there is a vehicle in the image or not. There are about 17k images of dimension (64x64x3).

Two labels:

Non-Vehicles
Vehicles

<h1 style="background-color:green;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Importing modules and Loading datasets</h1>
<a id=1></a>

In [ ]:
import  pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import keras 
from glob import glob
from keras.models import Sequential
from keras.layers import BatchNormalization,MaxPool2D,Dense,Conv2D,Flatten
from keras.callbacks import EarlyStopping,LearningRateScheduler
import warnings
from tqdm import tqdm
from PIL import Image
warnings.filterwarnings('ignore')

In [ ]:
df=pd.DataFrame(columns=['image','label'])


In [ ]:

# for a picture containing image of vehicle label is 1.

idx=0
for name in tqdm(glob('../input/vehicle-detection-image-set/data/vehicles/*')):
    image=Image.open(name)
    img_seq=image.getdata()
    img=np.array(img_seq)
    img=np.reshape(img,(64,64,3))
    data=pd.DataFrame({'image':[img],'label':[1]},index=[idx])
    df=df.append(data)    
    idx+=1

In [ ]:
# for a picture doesn't contain image of vehicle label is 0.

for name in tqdm(glob('../input/vehicle-detection-image-set/data/non-vehicles/*')):
    image=Image.open(name)
    img_seq=image.getdata()
    img=np.array(img_seq)
    img=np.reshape(img,(64,64,3))
    data=pd.DataFrame({'image':[img],'label':[0]},index=[idx]) 
    df=df.append(data)
    idx+=1

<h1 style="background-color:green;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Train test spliting of data</h1>
<a id=2></a>

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit,train_test_split
split=StratifiedShuffleSplit(n_splits=1,test_size=0.15,random_state=42)
for train_index,test_index in split.split(df,df['label']):
    train_data=df.iloc[train_index]
    test_data=df.iloc[test_index]
# train_x,test_x,train_y,test_y=train_test_split(df['image'],df['label'],random_state=42,test_size=0.15)

In [ ]:
train_img,train_label=train_data['image'],train_data['label']
test_img,test_label=test_data['image'],test_data['label']

<h1 style="background-color:green;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Data Visualization</h1>
<a id=3></a>

In [ ]:
plt.figure(figsize=(15,15))
for i in range(9):
    plt.subplot(3,3,(i%9)+1)
    ix=np.random.randint(3000)
    if(train_label.iloc[ix]==1):
        plt.title('Image no.{0} has vehicle'.format(i+1))
    elif(train_label.iloc[ix]==0):
        plt.title('Image no.{0} doesnot have vehicle'.format(i))
    plt.axis('off')
    plt.imshow(train_img.iloc[ix])
plt.show()

<h1 style="background-color:green;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Changing Dimension Of Data</h1>
<a id=4></a>

In [ ]:
def change_image_dimension(data):
    data=np.reshape(data.to_list(),(len(data),64,64,3))
    return data

In [ ]:
train_img=change_image_dimension(train_img)
test_img=change_image_dimension(test_img)
# train_img
train_img=train_img/255.0
test_img=test_img/255.0

In [ ]:
train_label=np.array(train_label,dtype=int)
test_label=np.array(test_label,dtype=int)

<h1 style="background-color:green;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Callback Functions</h1>
<a id=5></a>

In [ ]:
def schedule(epoch,lr):
    if epoch>=5:
        return 0.0001
    return 0.001
early_stop=EarlyStopping(monitor='val_accuracy',patience=3)
learning_rate_scheduler=LearningRateScheduler(schedule)

<h1 style="background-color:green;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Convolutional Neural Network</h1>
<a id=5></a>

In [ ]:
model=Sequential()
model.add(Conv2D(128,(3,3),activation='relu',input_shape=(64,64,3)))
model.add(MaxPool2D((2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
r=model.fit(train_img,train_label,validation_data=(test_img,test_label),epochs=10,batch_size=100,callbacks=[early_stop,learning_rate_scheduler])

In [ ]:
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])

In [ ]:
plt.plot(r.history['accuracy'])
plt.plot(r.history['val_accuracy'])

<h1 style="background-color:green;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Visualization of Predicted Data</h1>
<a id=3></a>

In [ ]:
plt.figure(figsize=(20,15))
for i in range(9):
    plt.subplot(3,3,(i%9)+1)
    rd_idx=np.random.randint(700)
    pred_test=model.predict_classes(np.reshape(test_img[rd_idx],(-1,64,64,3)))
    if pred_test==1:
        plt.title('Predicted that there is vehicle in Image')
    else:
        plt.title('Predicted thet there is no vehicle in Image')
    plt.axis('off')
    plt.imshow(test_img[rd_idx])
plt.show()


<h1 style="background-color:green;font-family:newtimeroman;font-size:400%;text-align:center;border-radius: 15px 50px;padding: 3px">Conclusion</h1>
<a id=3></a>

### Here we find that the model works quite well with accuracy of 99.99% on training dataset and 99.62% on test dataset.Thus this model is apt to identify the vehicles.

<h1 style="color:red;font-family:algerian;font-size:200%;text-align:center;border-radius: 15px 15px;padding: 3px">If You like the notebook, don't forget to upvote it. And if you have any doubt feel free to Ask in comment section.</h1>
<a id=3></a>

In [ ]:
nan